In [1]:
import pandas as pd
import os
import numpy as np

In [5]:
geneinfo = pd.read_table('../GSE92742_Broad_LINCS_gene_info.txt',index_col = 0)
# validgenes = validgenes = pd.read_table('../../../Ontology_Info/HGNC protein-coding_gene.txt',dtype = str,index_col = 0).symbol.tolist()
validgenes = geneinfo.pr_gene_symbol.unique().tolist()
files = os.listdir()
deg = pd.DataFrame(index = list(range(len(validgenes))))
fc = 1
files = [x for x in files if ('GSE' in x or 'gse' in x)]
for fn in files:    
    if fn[-4:] == 'xlsx':
        df = pd.read_excel(fn,index_col = 0)
    else:
        df = pd.read_table(fn,index_col = 0)
    df = df.loc[[x for x in df.index if x in validgenes]]
    dataset_id = fn.split('/')[-1].split("_")[0]
    deg_up = df[(df.logFC>=1)&(df.ix[:,'adj.P.Val']<=0.05)].sort_values('logFC',ascending = False).index.tolist()[:150]
    deg_dn = df[(df.logFC<=-1)&(df.ix[:,'adj.P.Val']<=0.05)].sort_values('logFC').index.tolist()[:150]
    deg.ix[list(range(len(deg_up))),dataset_id + '_up'] = sorted(deg_up)
    deg.ix[list(range(len(deg_dn))),dataset_id + '_dn'] = sorted(deg_dn)
deg = deg.dropna(how='all')
deg.to_excel('Clue Query Top 150 genes.xlsx')
# deg.to_excel('All FC1 pval005 protein coding DEG.xlsx')

In [4]:
gene_statistics = deg.stack()
gene_statistics = gene_statistics.reset_index()
network = pd.DataFrame(gene_statistics.set_index(0)['level_1'])
gene_statistics['reg'] = [x.split('_')[1] for x in gene_statistics.level_1]
gene_statistics = gene_statistics.groupby(0).reg.value_counts()
up_reg_genes = gene_statistics.loc[gene_statistics.index.get_level_values('reg') == 'up'].unstack()
dn_reg_genes = gene_statistics.loc[gene_statistics.index.get_level_values('reg') == 'dn'].unstack()
gene_statistics = pd.concat([up_reg_genes,dn_reg_genes],axis =1).fillna(0)
gene_statistics.columns = ['# of up_reg_datasets','# of dn_reg_datasets']
# gene_statistics.to_excel('All FC1 pval005 protein coding DEG summary statistics.xlsx')
network.columns = ['dataset']
network['regulation'] = [x.split('_')[1] for x in network.dataset]
network = network.join(gene_statistics,how='outer')
network['# of nondeg datasets'] = len(files) - network.ix[:,2] - network.ix[:,3]
network.to_excel('All FC1 pval005 protein coding DEG network.xlsx')

In [2]:
os.chdir('Intermediate results/Clue query')
files = os.listdir()
files  = [x for x in files if '150' in x]
cmpdlist = []
for fn in files:
    clue_tmp = pd.read_table(fn,index_col = 4)
    clue_tmp = clue_tmp[clue_tmp.Type =='cp']
    clue_tmp.Score = 100*clue_tmp.Score/clue_tmp.Score.max()
    cmpd_tmp = clue_tmp[clue_tmp.Score >= 90].index.tolist()
    cmpdlist+=cmpd_tmp

In [43]:
# os.chdir('../../Results/')
dn = pd.read_table('./LincDnEnrichmentIPF.gct',index_col = 0, skiprows=2)
up = pd.read_table('./LincUpEnrichmentIPF.gct',index_col = 0, skiprows=2)
col_2 = dn.columns[:2].tolist()
col_rest_dn = [x+'_dn' for x in dn.columns[2:]]
col_rest_up = [x+'_up' for x in up.columns[2:]]
dn.columns = col_2+col_rest_dn
up.columns = col_2+col_rest_up